In [1]:
import warnings
warnings.filterwarnings('ignore')

%load_ext rpy2.ipython

In [74]:
%%capture
%%R


In [75]:
%%R  -w 800 -h 1200
library(phytools)
library(paco)
library(plotly)
library(viridis)
library(svglite)

# read metadata table
data <- read.csv("/home/azureuser/datadrive/saccharopolyspora_dataset/data/processed/Staphylobactin_bigfam/tables/df_gtdb_meta.csv")

# preprocess genome tree
tree_genome <- read.tree("/home/azureuser/datadrive/saccharopolyspora_dataset/data/processed/Staphylobactin_bigfam/automlst_wrapper/final_corrected.newick")
# midpoint root
tree_genome <- phangorn::midpoint(tree_genome)
tree_genome <- ladderize(reorder(tree_genome))

# preprocess bgc tree
tree_bgc <- read.tree("/home/azureuser/datadrive/saccharopolyspora_dataset/data/processed/Staphylobactin_bigfam_bgc_filtered/notebooks/assets/data/bgc_tree_with_genome_id.tree")
# midpoint root
tree_bgc <- phangorn::midpoint(tree_bgc)
tree_bgc <- ladderize(reorder(tree_bgc))

# build links, here it mapped the same genome_ids
links <- cbind(tree_bgc$tip, tree_bgc$tip)
obj <- cophylo(tree_genome, tree_bgc, links, rotate = TRUE)

# Create a unique color for each family category.
family_levels <- unique(data$Family)
colors <- rainbow(length(family_levels))

# Create a named vector of colors for each family.
family_colors <- setNames(colors, family_levels)

# Assuming 'tree_bgc$tip.label' can be matched to 'data$tip_label' to find the genus.
tip_family1 <- data$Family[match(obj$trees[[1]]$tip.label, data$genome_id)]
tip_colors1 <- family_colors[tip_family1]

# Assuming 'tree_bgc$tip.label' can be matched to 'data$tip_label' to find the genus.
tip_family2 <- data$Family[match(obj$trees[[2]]$tip.label, data$genome_id)]
tip_colors2 <- family_colors[tip_family2]

# Assuming 'tree_bgc$tip.label' can be matched to 'data$tip_label' to find the genus.
link_family <- data$Family[match(links[,1], data$genome_id)]
link_colors <- family_colors[link_family]

#pdf("tanglegram.pdf", width=16, height=24)
svglite("output.svg", width = 16, height = 24)
# draw the cophylo
plot.cophylo(obj, link.type="curved", link.lwd=2, link.lty="solid", fsize=c(0.8,0.8), pts=FALSE,
             link.col = make.transparent(link_colors, 0.5))

nodelabels.cophylo(obj$trees[[1]]$node.label[2:Nnode(obj$trees[[1]])],
  2:Nnode(obj$trees[[1]])+Ntip(obj$trees[[1]]),frame="none",
  cex=0.5,adj=c(1,-0.4),which="left")

nodelabels.cophylo(obj$trees[[2]]$node.label[2:Nnode(obj$trees[[2]])],
  2:Nnode(obj$trees[[2]])+Ntip(obj$trees[[2]]),frame="none",
  cex=0.5,adj=c(0,1.4),which="right")

## add tip labels
tiplabels.cophylo(pch=21,frame="none",bg=tip_colors1,cex=1.5)
tiplabels.cophylo(pch=21,frame="none",bg=tip_colors2,which="right",cex=1.5)

# Use locator to interactively place the legend
# Note: Remove this line if you're running the script non-interactively or if you're saving to PDF
position <- locator(1)  # Click on the plot where you want the legend to appear

# Add a legend outside the plot area (to the right)
#par(xpd=TRUE)  # Allow plotting outside the plot area
legend(x="bottomleft", inset=c(-0.0, 0),  # Adjust inset to move the legend to the left
       legend=names(family_colors), 
       fill=family_colors, 
       title="Family", 
       cex=0.7, 
       pt.cex=0.7, 
       text.width=0.2)

dev.off()

Rotating nodes to optimize matching...
Done.
png 
  2 
